In [1]:
import query.datasets.prelude
reload(query.datasets.prelude)
from query.datasets.prelude import *
from query.datasets.tvnews.shot_detect import shot_detect, shot_stitch
from query.datasets.tvnews.face_detect import face_detect
from query.datasets.tvnews.face_embed import face_embed
from query.datasets.tvnews.pose_detect import pose_detect
from query.datasets.tvnews.identity_detect import identity_detect
from query.datasets.tvnews.animatedness import shot_frame_to_detect

In [2]:
with Timer('Detecting shots'):
    import query.datasets.tvnews.shot_detect
    reload(query.datasets.tvnews.shot_detect)
    from query.datasets.tvnews.shot_detect import shot_detect
    log.debug('Loading videos')
    all_videos = list(tqdm(Video.objects.all().order_by('id')))
    shot_indices, all_shots, all_blackframes = shot_detect(all_videos)
    shot_videos = gather(all_videos, shot_indices)
    log.debug('Computing face frames to detect')
    face_frame_per_shot = [[shot_frame_to_detect(shot) for shot in vid_shots]
                           for vid_shots in tqdm(all_shots)]

D 18-01-18 16:18:47 prelude.py:229] -- START: Detecting shots
D 18-01-18 16:18:47 <ipython-input-2-5028577b130d>:005] Loading videos


HBox(children=(IntProgress(value=0, max=62046), HTML(value=u'')))

D 18-01-18 16:18:48 prelude.py:229] -- START: Loading from cache: all_shots_blackframes


D 18-01-18 16:20:57 prelude.py:236] -- END: Loading from cache: all_shots_blackframes -- 00:02:05
D 18-01-18 16:20:57 <ipython-input-2-5028577b130d>:009] Computing face frames to detect


HBox(children=(IntProgress(value=0, max=60720), HTML(value=u'')))

D 18-01-18 16:21:25 prelude.py:236] -- END: Detecting shots -- 00:02:38


In [4]:
def bulk_create_copy(self, objects):
    meta = self.model._meta
    keys = [f.attname for f in meta._get_fields(reverse=False) 
            if not isinstance(f, models.ManyToManyField)]
    fname = '/app/rows.csv'
    log.debug('Creating CSV')
    with open(fname, 'wb') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(keys)
        max_id = self.all().aggregate(Max('id'))['id__max']
        id = max_id + 1 if max_id is not None else 0
        for obj in tqdm(objects):
            obj['id'] = id
            writer.writerow([obj[k] for k in keys])
            id += 1

    table = meta.db_table
    log.debug('Writing to database')
    with connection.cursor() as cursor:
        cursor.execute("COPY {} FROM '{}' DELIMITER ',' CSV HEADER".format(table, fname))
        cursor.execute("SELECT setval('{}_id_seq', {}, false)".format(table, id))

    os.remove(fname)
    log.debug('Done!')

In [4]:
with Timer('Saving shots'):
    Shot.objects.bulk_create_copy(flatten(all_shots))

D 18-01-18 16:23:42 prelude.py:229] -- START: Saving shots
D 18-01-18 16:23:45 prelude.py:345] Creating CSV


HBox(children=(IntProgress(value=0, max=60489549), HTML(value=u'')))

D 18-01-18 16:26:43 prelude.py:357] Writing to database


D 18-01-18 16:49:33 prelude.py:363] Done!
D 18-01-18 16:49:34 prelude.py:236] -- END: Saving shots -- 00:25:52


In [3]:
with Timer('Detecting sparse face'):
    import query.datasets.tvnews.face_detect
    reload(query.datasets.tvnews.face_detect)
    from query.datasets.tvnews.face_detect import face_detect
    
    all_faces, indices = face_detect(shot_videos, face_frame_per_shot)
    face_videos = gather(shot_videos, indices)
    face_shots = gather(all_shots, indices)
    face_frames = gather(face_frame_per_shot, indices)
    print(len(all_faces))

D 18-01-18 16:21:25 prelude.py:229] -- START: Detecting sparse face
D 18-01-18 16:21:25 prelude.py:229] -- START: Loading from cache: all_bboxes
D 18-01-18 16:22:33 prelude.py:236] -- END: Loading from cache: all_bboxes -- 00:01:01
D 18-01-18 16:22:33 prelude.py:236] -- END: Detecting sparse face -- 00:01:07


60607


In [7]:
sid = 72345948
for vid_shots in all_shots:
    for shot in vid_shots:
        shot['id'] = sid
        sid += 1

In [6]:
with Timer('Saving faces'):
#     log.debug('Creating frames')
#     frames_to_save = [
#         [{
#             'video_id': video.id,
#             'number': f
#         } for f in frames]
#         for (video, frames) in tqdm(zip(face_videos, face_frames))
#     ]
#     log.debug('Saving frames')
#     Frame.objects.bulk_create_copy(flatten(frames_to_save))

#     people_to_save = [
#         [[{'frame_id': frame['id']} for face in faces]
#          for (frame, faces) in zip(frames, vid_faces)]
#         for (frames, vid_faces) in tqdm(zip(frames_to_save, all_faces))
#     ]
#     Person.objects.bulk_create_copy(flatten(flatten(people_to_save)))
    
#     log.debug('Creating faces')
#     labeler, _ = Labeler.objects.get_or_create(name='mtcnn')
#     pid = 0
#     for (vid_faces, vid_shots) in tqdm(zip(all_faces, face_shots)):
#         for (frame_faces, shot) in zip(vid_faces, vid_shots):
#             for face in frame_faces:
#                 #face['frame_id'] = frame['id']
#                 face['labeler_id'] = labeler.id
#                 face['bbox_score'] = 1.0
#                 face['person_id'] = pid
#                 face['shot_id'] = shot['id']
#                 pid += 1
#     log.debug('Saving faces')
    Face.objects.bulk_create_copy(flatten(flatten(all_faces)))

D 18-01-18 20:11:56 prelude.py:229] -- START: Saving faces
D 18-01-18 20:12:04 prelude.py:345] Creating CSV


HBox(children=(IntProgress(value=0, max=43789933), HTML(value=u'')))

D 18-01-18 20:18:09 prelude.py:357] Writing to database
D 18-01-18 20:46:01 prelude.py:363] Done!
D 18-01-18 20:46:02 prelude.py:236] -- END: Saving faces -- 00:34:06


In [21]:
dir(Face._meta.get_fields()[2])

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_check_backend_specific_checks',
 '_check_choices',
 '_check_db_index',
 '_check_deprecation_details',
 '_check_field_name',
 '_check_null_allowed_for_primary_keys',
 '_check_primary_key',
 '_clear_cached_lookups',
 '_description',
 '_error_messages',
 '_get_default',
 '_get_flatchoices',
 '_get_lookup',
 '_get_val_from_obj',
 '_unique',
 '_unregister_lookup',
 '_validators',
 '_verbose_name',
 'attname',
 'auto_created',
 'auto_creation_counter',
 'blank',
 'cached_col',
 'check',
 'choices',
 'class_lookups',
 'clean',
 'clone',
 'column',
 'concrete',
 'contribute_to_class',
 'creation_counter',
 'db_check',
 'db_col

In [7]:
def output_name(video, frames):
    return video.path + '_faces_' + str(hash(tuple(frames)))

face_tables = [output_name(video, frames) for video, frames in tqdm(zip(face_videos, face_frames))]

HBox(children=(IntProgress(value=0, max=60607), HTML(value=u'')))

In [ ]:
#make_montage(face_videos[0], face_frames[0], '/app/montage.jpg', bboxes=all_faces[0])

In [8]:
with Timer('Gender faces'):
    import query.datasets.tvnews.gender_detect
    reload(query.datasets.tvnews.gender_detect)
    from query.datasets.tvnews.gender_detect import gender_detect
    
    all_genders, indices = gender_detect(face_videos, face_frames, face_tables)

D 18-01-18 20:46:05 prelude.py:229] -- START: Gender faces
D 18-01-18 20:46:05 prelude.py:229] -- START: Loading from cache: all_genders
D 18-01-18 20:46:42 prelude.py:236] -- END: Loading from cache: all_genders -- 00:00:31
D 18-01-18 20:46:42 prelude.py:236] -- END: Gender faces -- 00:00:37


In [9]:
gender_videos = gather(face_videos, indices)
gender_faces = gather(all_faces, indices)
gender_frames = gather(face_frames, indices)

In [ ]:
with Timer('Pinging db'):
    db_faces = list(Face.objects.all().values('id', 'person__frame__video__id', 'person__frame__number', 'bbox_x1'))

In [ ]:
def collect(l, kfn):
    d = defaultdict(list)
    for x in l:
        d[kfn(x)].append(x)
    return dict(d)

d1 = {k: collect(f, itemgetter('person__frame__number')) for k, f in collect(db_faces, itemgetter('person__frame__video__id')).iteritems()}

In [10]:
genders_to_save = []
genders = {
    k: Gender.objects.get_or_create(name=k)[0]
    for k in ['M', 'F', 'U']
}
labeler, _ = Labeler.objects.get_or_create(name='rudecarnie')
EPSILON = 0.0001
for (video, vid_faces, vid_frames, vid_genders) in tqdm(zip(gender_videos, gender_faces, gender_frames, all_genders)):
    for (frame_faces, frame, frame_genders) in zip(vid_faces, vid_frames, vid_genders):
        for (face, (label, score)) in zip(frame_faces, frame_genders):
#             face_id = None
#             for face2 in d1[video.id][frame]:
#                 if abs(face['bbox_x1'] - face2['bbox_x1']) < EPSILON:
#                     face_id = face2['id']
#                     break
#             if face_id is None:
#                 print(d1[video.id][frame], face)
#                 assert False
            face_id = face['id']

            if score < 0.9:
                label = 'U'

            genders_to_save.append({
                'face_id': face_id,
                'gender_id': genders[label].id,
                'labeler_id': labeler.id
            })

print(len(genders_to_save))

HBox(children=(IntProgress(value=0, max=60607), HTML(value=u'')))


43789933


In [12]:
def bulk_create_copy(self, objects):
    meta = self.model._meta
    keys = [
        f.attname for f in meta._get_fields(reverse=False)
        if not isinstance(f, models.ManyToManyField)
    ]
    fname = '/app/rows.csv'
    log.debug('Creating CSV')
    with open(fname, 'wb') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(keys)
        max_id = self.all().aggregate(Max('id'))['id__max']
        id = max_id + 1 if max_id is not None else 0
        for obj in tqdm(objects):
            obj['id'] = id
            writer.writerow([obj[k] for k in keys])
            id += 1

    table = meta.db_table
    log.debug('Writing to database')
    with connection.cursor() as cursor:
        cursor.execute("COPY {} ({}) FROM '{}' DELIMITER ',' CSV HEADER".format(
            table, ', '.join(keys), fname))
        cursor.execute("SELECT setval('{}_id_seq', {}, false)".format(table, id))

    os.remove(fname)
    log.debug('Done!')

# genders_to_save2 = []
# seen = set()
# for g in tqdm(genders_to_save):
#     if g['face_id'] not in seen:
#         genders_to_save2.append(g)
#     seen.add(g['face_id'])
bulk_create_copy(FaceGender.objects, genders_to_save2)

D 18-01-18 21:03:15 <ipython-input-12-ff7c93f4fdfa>:008] Creating CSV


HBox(children=(IntProgress(value=0, max=43789933), HTML(value=u'')))

D 18-01-18 21:05:10 <ipython-input-12-ff7c93f4fdfa>:020] Writing to database


D 18-01-18 21:31:31 <ipython-input-12-ff7c93f4fdfa>:027] Done!


In [ ]:
def output_name(video, frames):
    return video.path + '_embeddings_' + str(hash(tuple(frames)))

with make_scanner_db() as db:
    db._load_db_metadata()
    
filter2_videos, filter2_frames, filter2_faces, filter2_shots, embed_tables = unzip(
    [(video, frames, faces, shots, db.table(output_name(video, frames)))
          for (video, frames, faces, shots) in tqdm(zip(filtered_videos, filtered_frames, filtered_faces, filtered_shots))
          if db.has_table(output_name(video, frames)) and db.table(output_name(video, frames)).committed()])


In [ ]:
with Timer("Embedding faces"):
    def load_embs():
        log.debug('Loading embs')
        EMBEDDING_SIZE = 128
        def load((t, vid_faces)):
            embs = list(t.column('embeddings').load())
            arrays = [np.frombuffer(emb, dtype=np.float32) if emb is not None else [] for _, emb in embs]
            return [np.split(a, len(a) / 128) if len(a) > 0 else [] for a in arrays]
        return par_for(load, zip(embed_tables, filter2_faces), workers=8)

    all_embs = pcache.get('all_embs', load_embs, method='pickle')


In [ ]:
import struct

serializers = {}
type_hashes = {}
def serializer(ty):
    def register_class(cls):
        serializers[ty] = cls
        type_hashes[hash(ty)] = ty
        return cls
    return register_class

def serialize(v):
    ty = type(v)
    if ty in serializers:
        serializer = serializers[ty]
        hsh = hash(ty)
    else:
        serializer = DefaultSerializer
        hsh = 0
    return '{}{}'.format(struct.pack('=i', hsh), serializer.serialize(v))

def _deserialize(s):
    hsh, s = struct.unpack('=i', s[:4])[0], s[4:]
    serializer = serializers[type_hashes[hsh]]
    return serializer.deserialize(s)

def deserialize(s):
    return _deserialize(s)[0]
    
class DefaultSerializer:
    @staticmethod
    def serialize(v):
        return serialize(pickle.dumps(v))
    
    @staticmethod
    def deserialize(s):
        ps, s = _deserialize(s)
        return pickle.loads(ps), s
    
type_hashes[0] = 0    
serializers[0] = DefaultSerializer    

@serializer(int)
class IntSerializer:
    @staticmethod
    def serialize(v):
        return struct.pack('=q', v)
    
    @staticmethod
    def deserialize(s):
        return struct.unpack('=q', s[:8])[0], s[8:]
    
@serializer(str)    
class StringSerializer:
    @staticmethod
    def serialize(v):
        return '{}{}'.format(serialize(len(v)), v)
    
    @staticmethod
    def deserialize(s):
        n, s = _deserialize(s)
        return s[:n], s[n:]

@serializer(np.array)    
class NumpySerializer:
    @staticmethod
    def serialize(v):
        dtype = serialize(pi)
        n = serialize(len(v))
        return '{}{}{}'.format(dtype, n, v.tobytes())
    
    @staticmethod
    def deserialize(s):
        dtype, s = _deserialize(s)
        n, s = _deserialize(s)
        return np.frombuffer(s[:n], dtype=dtype), s[n:]

@serializer(list)    
class ListSerializer:
    @staticmethod
    def serialize(v):
        n = serialize(len(v))
        return '{}{}'.format(
            n, ''.join([serialize(x) for x in v])
        )
    
    @staticmethod
    def deserialize(s):
        n, s = _deserialize(s)
        l = []
        for _ in range(n):
            x, s = _deserialize(s)
            l.append(x)
        return l, s

def test(x):
    assert(deserialize(serialize(x)) == x)

test(1)
test("hello")
test([1, "hello"])

# with Timer('myser'):
#     x = serialize(all_embs[:10])
# with Timer('mydeser'):
#     deserialize(x)
# with Timer('pickleser'):
#     x = pickle.dumps(all_embs[:10])
# with Timer('pickledeser'):
#     pickle.loads(x)

#print(len(pickle.dumps(all_embs[0])))
#print(sum([sum([e.nbytes for e in l]) for l in all_embs[0]]))


In [ ]:
with Timer("Stitching shots"):
    import query.datasets.prelude
    reload(query.datasets.prelude)
    def load_stitches():
        log.debug('Computing stitches')
        import query.datasets.tvnews.shot_detect
        reload(query.datasets.tvnews.shot_detect)
        return query.datasets.tvnews.shot_detect.shot_stitch(filter2_videos, filter2_shots, filter2_frames, filter2_faces, all_embs)
    (stitched_shots, stitched_indices) = query.datasets.prelude.pcache.get('stitched_shots', load_stitches)

In [ ]:
def gather(l, idx):
    return [l[i] for i in idx]

def gather2(l, idx):
    return [l[i][j] for i, j in idx]

(filter3_videos, filter3_frames, filter3_faces) = unzip([
    (video, gather(frames, idxs), gather(faces, idxs))
    for (video, frames, faces, idxs) in tqdm(zip(filter2_videos, filter2_frames, filter2_faces,  stitched_indices))
])

# (filter3_videos, filter3_frames, filter3_faces, filter3_embs) = unzip([
#     (video, gather(frames, idxs), gather(faces, idxs), gather(embs, idxs))
#     for (video, frames, faces, embs, idxs) in tqdm(zip(filter2_videos, filter2_frames, filter2_faces, all_embs, stitched_indices))
# ])

In [ ]:
show = Show.objects.get(name='The Rachel Maddow Show').id
indices = [i for i, video in enumerate(filter3_videos) if video.show_id == show]

with Timer('Detecting identities'):
    def load_identities():
        log.debug('Computing identities')
        import query.datasets.tvnews.identity_detect
        reload(query.datasets.tvnews.identity_detect)

        return query.datasets.tvnews.identity_detect.identity_detect(
           gather(filter3_videos, indices), "/app/rachel-maddow.jpg", gather(filter3_embs, indices))
    
    matching_indices = pcache.get('matching_indices', load_identities)
    
print(len(matching_indices), sum([len(l) for l in matching_indices]))

In [ ]:
matching_indices_onelevel = [[j for j, k in l] for l in matching_indices]
matching_videos, matching_shots, matching_frames, matching_faces = \
    (gather(filter3_videos, indices),      
    [gather(shots, idxs) for (shots, idxs) in zip(gather(stitched_shots, indices), matching_indices_onelevel)],
    [gather(frames, idxs) for (frames, idxs) in zip(gather(filter3_frames, indices), matching_indices_onelevel)],
    [gather2(faces, idxs) for (faces, idxs) in zip(gather(filter3_faces, indices), matching_indices)])
    
#pose_blacklist = ['tvnews/videos/MSNBCW_20170708_010000_The_Rachel_Maddow_Show.mp4']    
#pose_blacklist = ['tvnews/videos/MSNBCW_20170506_040000_The_Rachel_Maddow_Show.mp4']
pose_blacklist = []
    
matching2_videos, matching2_shots, matching2_frames, matching2_faces = unzip([
    (video, shots, frames, faces)
    for video, shots, frames, faces in zip(matching_videos, matching_shots, matching_frames, matching_faces)
    if len(shots) > 0 and video.path not in pose_blacklist
])
    
TARGET_FPS = 10
pose_frames = [
    sum([list(range(s['min_frame'], s['max_frame'], int(round(video.fps / TARGET_FPS)))) for s in shots], [])
    for (video, shots) in zip(matching2_videos, matching2_shots)    
]

In [ ]:
import query.datasets.tvnews.pose_detect
reload(query.datasets.tvnews.pose_detect)
from query.datasets.tvnews.pose_detect import pose_detect

all_poses = pose_detect(matching2_videos, pose_frames, force=True)

In [ ]:
import query.datasets.tvnews.pose_detect
reload(query.datasets.tvnews.pose_detect)
from query.datasets.tvnews.pose_detect import pose_track


#pose_track(matching2_videos, matching2_shots, matching2_frames, matching2_faces, all_poses)

In [ ]:
def bbox_montage((i, (video, frames, faces, matches))):
    if len(matches) == 0:
        print(video.path)
        return
    make_montage(
        video,
        [frames[j] for j, _ in matches],
        '/tmp/montage{}.jpg'.format(i),
        bboxes=[[faces[j][k]] for j, k in matches],
        progress=False)

In [ ]:
import query.datasets.prelude
reload(query.datasets.prelude)
from query.datasets.prelude import *

# make_montage(filter3_videos[indices[i]], filter3_frames[indices[i]],
#              '/app/montage.jpg', filter3_faces[indices[i]], workers=96, progress=True)

def bbox_montage((i, (video, frames, faces, matches))):
    try:
        if len(matches) == 0:
            print(video.path)
            return
        bbox_map = defaultdict(list, {j: [faces[j][k]] for j, k in matches})
        make_montage(
            video,
            #[frames[j] for j, _ in matches],
            frames,
            '/tmp/montage{}.jpg'.format(i),
            bboxes=[bbox_map[i] for i in range(len(frames))],
            #bboxes=[[faces[j][k]] for j, k in matches],
            progress=False)
    except Exception:
        traceback.print_exc()
        print(video.path)
    
_ = par_for(bbox_montage,
        list(enumerate(zip(gather(filter3_videos, indices)[:100], gather(filter3_frames, indices), 
            gather(filter3_faces, indices), matching_indices))),
        process=True,
        workers=8)

In [ ]:
import requests
import cv2

par_for(make_montage,
        list(enumerate(zip(filter2_videos, [[s['min_frame'] for s in l] for l in stitched_shots][:100]))),
        process=True,
        workers=8)

In [ ]:
Shot.objects.bulk_create_copy({
    'min_frame': shot['min_frame'], 
    'max_frame': shot['max_frame'], 
    'labeler_id': shot['labeler'],
    'video_id': shot['video__id']
} for shot_list in tqdm(stitched_shots) for shot in shot_list])